# 成人收入数据集 BT-TWD 可行性实验

本 notebook 按步骤运行：加载配置 → 读取数据 → 预处理 → 桶树划分 → 基线与 BTTWD k 折实验 → 桶级分析。

In [1]:
# 步骤0：环境与路径设置
import os, sys
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
plt.rcParams['axes.unicode_minus'] = False

# 将项目根目录加入路径，便于导入 bttwdlib
root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
if root_path not in sys.path:
    sys.path.append(root_path)

from bttwdlib import (
    load_yaml_cfg,
    show_cfg,
    load_adult_raw,
    prepare_features_and_labels,
    BucketTree,
    run_kfold_experiments,
    log_info,
    set_global_seed,
)

cfg_path = os.path.join(root_path, 'configs', 'adult_bttwd.yaml')
cfg = load_yaml_cfg(cfg_path)
set_global_seed(cfg.get('SEED', {}).get('global_seed', 42))
log_info('【步骤0摘要】环境准备完毕，路径与随机种子已设置。')

【INFO】【2025-11-23 15:35:50】【配置加载】已读取 e:\yan\组\三支决策\机器学习\BT_TWD\configs\adult_bttwd.yaml
【INFO】【2025-11-23 15:35:53】【步骤0摘要】环境准备完毕，路径与随机种子已设置。


In [2]:
# 步骤1：加载配置
show_cfg(cfg)
log_info('【步骤1摘要】配置文件加载完成，关键参数检查通过。')

【INFO】【2025-11-23 15:35:53】【配置-数据】数据集=adult, k折=5, 目标列=income, 正类=">50K"
【INFO】【2025-11-23 15:35:53】【配置-BTTWD】阈值模式=None, 全局模型=xgb, 桶内模型=knn, 后验估计器(兼容字段)=logreg
【INFO】【2025-11-23 15:35:53】【配置-基线】LogReg启用=True, RandomForest启用=True, KNN启用=True, XGBoost启用=True
【INFO】【2025-11-23 15:35:53】【步骤1摘要】配置文件加载完成，关键参数检查通过。


In [3]:
# 步骤2：加载原始数据
df_raw = load_adult_raw(cfg)
display(df_raw.head())
target_col = cfg['DATA']['target_col']
class_counts = df_raw[target_col].value_counts(normalize=True)
ax = class_counts.plot(kind='bar', title='正负类比例')
plt.ylabel('比例')
fig_path = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'class_distribution.png')
os.makedirs(os.path.dirname(fig_path), exist_ok=True)
plt.savefig(fig_path, bbox_inches='tight')
plt.close()
log_info('【步骤2摘要】Adult 原始数据加载与基本统计完成。')

【INFO】【2025-11-23 15:35:53】【数据加载完毕】样本数=32561，特征数=14，正类比例=0.24


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


【INFO】【2025-11-23 15:35:53】【步骤2摘要】Adult 原始数据加载与基本统计完成。


In [4]:
# 步骤3：预处理与特征工程
X, y, meta = prepare_features_and_labels(df_raw, cfg)
log_info(f'【预处理】编码特征维度={X.shape[1]}，样本数={X.shape[0]}')
log_info(f"【步骤3摘要】特征预处理完成：连续={len(meta['continuous_cols'])}，类别={len(meta['categorical_cols'])}，编码维度={X.shape[1]}。")

【INFO】【2025-11-23 15:35:53】【预处理】连续特征=6个，类别特征=8个
【INFO】【2025-11-23 15:35:53】【预处理】编码后维度=100
【INFO】【2025-11-23 15:35:53】【预处理】编码特征维度=100，样本数=32561
【INFO】【2025-11-23 15:35:53】【步骤3摘要】特征预处理完成：连续=6，类别=8，编码维度=100。


In [5]:
# 步骤4：构建桶树并检查划分
bucket_tree = BucketTree(cfg['BTTWD']['bucket_levels'], feature_names=df_raw.drop(columns=[cfg['DATA']['target_col']]).columns.tolist())
bucket_ids_full = bucket_tree.assign_buckets(df_raw.drop(columns=[cfg['DATA']['target_col']]))
bucket_df = bucket_ids_full.value_counts().reset_index()
bucket_df.columns = ['bucket_id', 'count']
bucket_df['pos_rate'] = df_raw.groupby(bucket_ids_full)[cfg['DATA']['target_col']].apply(lambda s: (s == cfg['DATA']['positive_label']).mean()).values
display(bucket_df.head())
bucket_df.set_index('bucket_id')['count'].plot(kind='bar', figsize=(12,4), title='桶样本数分布')
fig_bucket = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'bucket_metrics_bar.png')
plt.savefig(fig_bucket, bbox_inches='tight')
plt.close()
log_info(f'【步骤4摘要】桶树划分完成，共有 {bucket_ids_full.nunique()} 个叶子桶。')

【INFO】【2025-11-23 15:35:53】【桶树】已为样本生成桶ID，共 16 个组合


,bucket_id,count,pos_rate
0,L1_age=old|L2_education=high,5328,0.235657
1,L1_age=mid|L2_education=high,4218,0.018779
2,L1_age=old|L2_education=mid,4166,0.097167
3,L1_age=mid|L2_education=mid,3530,0.452080
4,L1_age=young|L2_education=high,3216,0.411974


【INFO】【2025-11-23 15:35:54】【步骤4摘要】桶树划分完成，共有 16 个叶子桶。


In [6]:
# 步骤5：运行基线模型 k 折实验
# 基线部分在 run_kfold_experiments 内统一调度
log_info('【步骤5】基线模型将在整体交叉验证中一并运行。')
log_info('【步骤5摘要】基线模型性能将作为后续对比基准。')

【INFO】【2025-11-23 15:35:54】【步骤5】基线模型将在整体交叉验证中一并运行。
【INFO】【2025-11-23 15:35:54】【步骤5摘要】基线模型性能将作为后续对比基准。


In [7]:
# 步骤6：运行 BTTWD k 折实验（含基线）
results = run_kfold_experiments(X, y, df_raw.drop(columns=[cfg['DATA']['target_col']]), cfg)
summary_df = pd.read_csv(os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'metrics_kfold_summary.csv'))
display(summary_df)
summary_df.plot(x='model', kind='bar', figsize=(8,4), title='模型指标对比')
fig_compare = os.path.join(root_path, cfg['OUTPUT']['figs_dir'], 'metrics_compare.png')
plt.savefig(fig_compare, bbox_inches='tight')
plt.close()
log_info('【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。')

【INFO】【2025-11-23 15:36:12】【基线-LogReg】整体指标：Precision=0.735, Recall=0.602, F1=0.662, BAC=0.767, AUC=0.907, MCC=0.573, Kappa=0.568
【INFO】【2025-11-23 15:36:39】【基线-RF】整体指标：Precision=0.739, Recall=0.627, F1=0.678, BAC=0.778, AUC=0.906, MCC=0.590, Kappa=0.587
【INFO】【2025-11-23 15:36:43】【基线-KNN】整体指标：Precision=0.711, Recall=0.525, F1=0.604, BAC=0.729, AUC=0.869, MCC=0.512, Kappa=0.502


C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [15:36:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [15:36:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [15:36:46] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.



【INFO】【2025-11-23 15:36:48】【基线-XGB】整体指标：Precision=0.786, Recall=0.655, F1=0.714, BAC=0.799, AUC=0.929, MCC=0.639, Kappa=0.634
【INFO】【2025-11-23 15:36:48】【K折实验】正在执行第 1/5 折...
【INFO】【2025-11-23 15:36:48】【桶树】已为样本生成桶ID，共 16 个组合
【INFO】【2025-11-23 15:36:49】【BTTWD】桶 L1_age=mid|L2_education=high 向父桶 L1_age=mid 贡献 681 个典型样本
【INFO】【2025-11-23 15:36:49】【BTTWD】桶 L1_age=mid|L2_education=low 向父桶 L1_age=mid 贡献 50 个典型样本
【INFO】【2025-11-23 15:36:49】【BTTWD】桶 L1_age=mid|L2_education=mid 向父桶 L1_age=mid 贡献 561 个典型样本
【INFO】【2025-11-23 15:36:49】【BTTWD】桶 L1_age=mid|L2_education=top 向父桶 L1_age=mid 贡献 87 个典型样本
【INFO】【2025-11-23 15:36:49】【BTTWD】桶 L1_age=old|L2_education=high 向父桶 L1_age=old 贡献 854 个典型样本
【INFO】【2025-11-23 15:36:49】【BTTWD】桶 L1_age=old|L2_education=low 向父桶 L1_age=old 贡献 54 个典型样本
【INFO】【2025-11-23 15:36:49】【BTTWD】桶 L1_age=old|L2_education=mid 向父桶 L1_age=old 贡献 667 个典型样本
【INFO】【2025-11-23 15:36:49】【BTTWD】桶 L1_age=old|L2_education=top 向父桶 L1_age=old 贡献 217 个典型样本
【INFO】【2025-11-23 15:36:49】【BTTWD】桶 L1_ag

C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [15:36:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-11-23 15:36:49】【BTTWD】全局模型训练完成，用于兜底预测
【INFO】【2025-11-23 15:36:53】【BTTWD】叶子桶 L1_age=young|L2_education=low 训练样本不足或单类，跳过局部模型训练
【INFO】【2025-11-23 15:36:55】【BTTWD】共生成 16 个叶子桶，其中有效桶 18 个（样本数 ≥ 100）
【INFO】【2025-11-23 15:36:55】【桶树】已为样本生成桶ID，共 16 个组合
【INFO】【2025-11-23 15:36:55】【桶树】已为样本生成桶ID，共 16 个组合
【INFO】【2025-11-23 15:36:55】【桶树】已为样本生成桶ID，共 16 个组合
【INFO】【2025-11-23 15:36:55】【BTTWD】本折指标：Precision=0.600, Recall=0.713, F1=0.652, BAC=0.781, AUC=0.872, MCC=0.532, Kappa=0.528
【INFO】【2025-11-23 15:36:55】【桶树】已为样本生成桶ID，共 16 个组合
【INFO】【2025-11-23 15:36:55】【K折实验】正在执行第 2/5 折...
【INFO】【2025-11-23 15:36:56】【桶树】已为样本生成桶ID，共 16 个组合
【INFO】【2025-11-23 15:36:56】【BTTWD】桶 L1_age=mid|L2_education=high 向父桶 L1_age=mid 贡献 672 个典型样本
【INFO】【2025-11-23 15:36:56】【BTTWD】桶 L1_age=mid|L2_education=low 向父桶 L1_age=mid 贡献 50 个典型样本
【INFO】【2025-11-23 15:36:56】【BTTWD】桶 L1_age=mid|L2_education=mid 向父桶 L1_age=mid 贡献 564 个典型样本
【INFO】【2025-11-23 15:36:56】【BTTWD】桶 L1_age=mid|L2_education=top 向父桶 L1_age=mid 贡献 87 个典型样本
【INFO

...

【INFO】【2025-11-23 15:37:10】【BTTWD】全局模型训练完成，用于兜底预测
【INFO】【2025-11-23 15:37:15】【BTTWD】叶子桶 L1_age=young|L2_education=low 训练样本不足或单类，跳过局部模型训练
【INFO】【2025-11-23 15:37:16】【BTTWD】共生成 16 个叶子桶，其中有效桶 18 个（样本数 ≥ 100）
【INFO】【2025-11-23 15:37:16】【桶树】已为样本生成桶ID，共 16 个组合
【INFO】【2025-11-23 15:37:17】【桶树】已为样本生成桶ID，共 16 个组合
【INFO】【2025-11-23 15:37:17】【桶树】已为样本生成桶ID，共 16 个组合
【INFO】【2025-11-23 15:37:17】【BTTWD】本折指标：Precision=0.592, Recall=0.726, F1=0.652, BAC=0.783, AUC=0.873, MCC=0.531, Kappa=0.526
【INFO】【2025-11-23 15:37:17】【桶树】已为样本生成桶ID，共 16 个组合
【INFO】【2025-11-23 15:37:17】【K折实验】正在执行第 5/5 折...
【INFO】【2025-11-23 15:37:17】【桶树】已为样本生成桶ID，共 16 个组合
【INFO】【2025-11-23 15:37:17】【BTTWD】桶 L1_age=mid|L2_education=high 向父桶 L1_age=mid 贡献 680 个典型样本
【INFO】【2025-11-23 15:37:17】【BTTWD】桶 L1_age=mid|L2_education=low 向父桶 L1_age=mid 贡献 50 个典型样本
【INFO】【2025-11-23 15:37:17】【BTTWD】桶 L1_age=mid|L2_education=mid 向父桶 L1_age=mid 贡献 555 个典型样本
【INFO】【2025-11-23 15:37:17】【BTTWD】桶 L1_age=mid|L2_education=top 向父桶 L1_age=mid 贡献 91 个典型样本
【INFO

C:\Users\xzq20.HUANLI\AppData\Roaming\Python\Python311\site-packages\xgboost\core.py:158: UserWarning: [15:37:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


【INFO】【2025-11-23 15:37:17】【BTTWD】全局模型训练完成，用于兜底预测
【INFO】【2025-11-23 15:37:22】【BTTWD】叶子桶 L1_age=young|L2_education=low 训练样本不足或单类，跳过局部模型训练
【INFO】【2025-11-23 15:37:23】【BTTWD】共生成 16 个叶子桶，其中有效桶 18 个（样本数 ≥ 100）
【INFO】【2025-11-23 15:37:23】【桶树】已为样本生成桶ID，共 16 个组合
【INFO】【2025-11-23 15:37:24】【桶树】已为样本生成桶ID，共 16 个组合
【INFO】【2025-11-23 15:37:24】【桶树】已为样本生成桶ID，共 16 个组合
【INFO】【2025-11-23 15:37:24】【BTTWD】本折指标：Precision=0.597, Recall=0.739, F1=0.660, BAC=0.790, AUC=0.877, MCC=0.543, Kappa=0.537
【INFO】【2025-11-23 15:37:24】【桶树】已为样本生成桶ID，共 16 个组合
【INFO】【2025-11-23 15:37:24】【K折实验】所有结果已写入 results 目录


,model,Precision_mean,Precision_std,Recall_mean,Recall_std,F1_mean,F1_std,BAC_mean,BAC_std,AUC_mean,...,POS_Coverage_std,Regret_mean,Regret_std,Precision,Recall,F1,BAC,AUC,MCC,Kappa
0,BTTWD,0.596554,0.006709,0.735751,0.020932,0.658711,0.006687,0.788911,0.007099,0.876657,...,0.010807,0.305089,0.010041,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,LogReg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.735088,0.601964,0.661899,0.766577,0.906751,0.572938,0.568290
2,RandomForest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.738954,0.627088,0.678441,0.778411,0.905658,0.590440,0.587168
3,KNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.711249,0.524933,0.604050,0.728668,0.868943,0.511519,0.502254
4,XGBoost,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.785889,0.654891,0.714435,0.799149,0.929218,0.638742,0.634399


【INFO】【2025-11-23 15:37:25】【步骤6摘要】BTTWD 与基线的 k 折结果已生成并保存。


In [8]:
# 步骤7：桶级别分析
bucket_metrics_path = os.path.join(root_path, cfg['OUTPUT']['results_dir'], 'bucket_metrics.csv')
if os.path.exists(bucket_metrics_path):
    bucket_metrics_df = pd.read_csv(bucket_metrics_path)
    display(bucket_metrics_df.head())
    bucket_metrics_df.plot(x='bucket_id', y='pos_rate_all', kind='bar', figsize=(12,4), title='桶正类比例')
    plt.ylabel('正类比例')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.savefig(fig_bucket, bbox_inches='tight')
    plt.close()
log_info('【步骤7摘要】桶级指标已整理，可用于局部化分析。')

,bucket_id,layer,parent_bucket_id,n_train,n_val,pos_rate_train,pos_rate_val,alpha,beta,regret_val,...,use_parent_threshold,threshold_n_samples,n_all,pos_rate_all,n_test,pos_rate_test,BND_ratio_test,POS_Coverage_test,regret_test,fold
0,L1_age=old|L2_education=high,L2,L1_age=old,2992,1280,0.408422,0.422656,0.35,0.2,0.380078,...,False,1280,4272,0.412687,1056.0,0.409091,0.0,0.550189,0.404830,1
1,L1_age=mid|L2_education=high,L2,L1_age=mid,2416,990,0.235099,0.243434,0.35,0.1,0.368182,...,False,990,3406,0.237522,812.0,0.227833,0.0,0.268473,0.375616,1
2,L1_age=old|L2_education=mid,L2,L1_age=old,2344,991,0.213737,0.211907,0.45,0.2,0.414733,...,False,991,3335,0.213193,831.0,0.216606,0.0,0.217810,0.421179,1
3,L1_age=mid|L2_education=mid,L2,L1_age=mid,1925,884,0.092987,0.097285,0.55,0.5,0.263575,...,False,884,2809,0.094340,721.0,0.108183,0.0,0.008322,0.310680,1
4,L1_age=young|L2_education=high,L2,L1_age=young,1799,753,0.026126,0.018592,0.45,0.1,0.044489,...,False,753,2552,0.023903,664.0,0.024096,0.0,0.004518,0.073795,1


【INFO】【2025-11-23 15:37:26】【步骤7摘要】桶级指标已整理，可用于局部化分析。


In [9]:
# 步骤8：结果汇总
log_info('【步骤8】检查结果文件与图表。')
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['results_dir'])))
print(os.listdir(os.path.join(root_path, cfg['OUTPUT']['figs_dir'])))
log_info('【全部步骤完成】Adult 数据集上的 BT-TWD 可行性实验结束。')

【INFO】【2025-11-23 15:37:26】【步骤8】检查结果文件与图表。
['bucket_metrics.csv', 'bucket_thresholds_per_fold.csv', 'metrics_kfold_per_fold.csv', 'metrics_kfold_summary.csv']
['bucket_metrics_bar.png', 'class_distribution.png', 'metrics_compare.png']
【INFO】【2025-11-23 15:37:26】【全部步骤完成】Adult 数据集上的 BT-TWD 可行性实验结束。
